# Analysis

In [ ]:
import pandas as pd
pd.set_option("display.max_columns",100)

In [ ]:
data = pd.read_csv('../data/train_set.csv')

In [ ]:
data.head()

In [ ]:
data.shape

# Train model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X = data.copy()
y = X.pop('label')

In [ ]:
model = LogisticRegression()
model.fit(X[['user_age', 'body_length', 'channels']], y)

In [ ]:
X[['user_age', 'body_length', 'channels']].dtypes

### Saving best model as pickle

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, '../api/model.pkl') # other folder eventually

In [ ]:
clf = joblib.load('model.pkl') 

In [ ]:
testcase = pd.DataFrame(X.loc[0, ['user_age', 'body_length', 'channels']]).T

In [ ]:
clf.predict(testcase)

In [ ]:
clf.predict_proba(testcase)[0, 1]